In [2]:
import json
# count
with open('EEG_analysis/psd_filt.json') as f:
    psd = json.load(f) #{path:[1*25]}

import pandas as pd
iid2item = {1: 'SOBNXDP12A6D4F8880', 2: 'SOJSHBN12A8C138AAC', 3: 'SOGPBAW12A6D4F9F22', 4: 'SOICNON12A8C140437', 5: 'SOAOAHZ12A8C13AAF1', 6: 'SONGOJV12AF729AEBD', 7: 'SOVKSNI12AB018A15B', 8: 'SOZYNNT12A81C22E0F', 9: 'SOPXXDD12AB0184A81', 10: 'SOXMZYS12AB018349B', 11: 'SOZVILY12AB01855ED', 12: 'SOVSELF12A670215EB', 13: 'SOITRTA12A6D4F8261', 14: 'SODLCIU12A8AE45F1D', 15: 'SOOTKWZ12AB0181082', 16: 'SOEBUPS12AF729F765', 17: 'SOWQQSX12A58A7B81D', 18: 'SOTWSMJ12A6D4FB9DF', 19: 'SOWDLPO12A6D4F72BB', 20: 'SOCYOTN12A8C13A4B1', 21: 'SOUHNQN12AF72A3DE3', 22: 'SOEWPQB12A8C13222B', 23: 'SOZDTTJ12AB0183DFE', 24: 'SORWJSF12A8C138AB6', 25: 'SOQAIAY12AB017B7AC'}
item2iid = {iid2item[iid]:iid for iid in list(iid2item.keys())}
item_df = pd.read_csv('/work/hzy/EEG_music/cut_data/item_mid.csv','\t')

iid_data_dict = {}
for i in range(len(item_df)):
    iid = item_df['i_id_c'][i]
    if iid == 0:
        continue
    # i_popularity_i	i_year_c	i_danceability_f	i_energy_f	i_loudness_f	i_speechiness_f	i_acousticness_f	i_instrumentalness_f	i_liveness_f	i_valence_f	i_tempo_f
    iid_data_dict[iid] = [item_df['i_popularity_i'][i],item_df['i_year_c'][i],item_df['i_danceability_f'][i],item_df['i_energy_f'][i],
                        item_df['i_loudness_f'][i],item_df['i_speechiness_f'][i],item_df['i_acousticness_f'][i],item_df['i_instrumentalness_f'][i],
                        item_df['i_liveness_f'][i],item_df['i_valence_f'][i],item_df['i_tempo_f'][i]]

label_df = pd.read_csv('cut_data/label.csv','\t',usecols=(1,2,3,4,5,6,7))

import numpy as np
audio_all=[] # 由11个dict组成，每个dict是{path:1}
for i in range(11):
    audio_all.append({})
for i in range(len(label_df)):
    path = label_df['path'][i]
    item = label_df['echo_id'][i]
    audio_list = iid_data_dict[item2iid[item]]
    for j in range(11):
        audio_all[j][path]=audio_list[j]
# print(audio_all)

# for preference
print(label_df['rating'].value_counts())
preference = list(label_df['rating'])
a_mean = np.mean(preference)
a_var = np.var(preference)
a_std = np.std(preference,ddof=1)
print(a_mean,a_var,a_std)

label={}
for i in range(len(label_df)):
    path = label_df['path'][i]
    label[path]=label_df['rating'][i]

3    112
4     77
2     62
1     35
5     33
Name: rating, dtype: int64
3.0344827586206895 1.2872121932764022 1.1363362312107175


In [ ]:
from scipy import stats
audio_feature = ['i_popularity_i','i_year_c','i_danceability_f','i_energy_f','i_loudness_f','i_speechiness_f',
                 'i_acousticness_f','i_instrumentalness_f','i_liveness_f','i_valence_f','i_tempo_f']

threshold =[[43,69],[1996,2007],[0.518,0.518],[0.698,0.887],[-6.108,-6.108]]
for i in range(11):
    print('------------\n',audio_feature[i])
    audio_list= []
    psd_arr = np.zeros((319,25))
    lab_arr = np.zeros((319))
    
    dic_audio = sorted(audio_all[i].items(), key=lambda x: x[1])
    j=0
    for tup in dic_audio:
        psd_arr[j,:] = np.array(psd[tup[0]])
        lab_arr[j] = label[tup[0]]
        
        audio_list.append(tup[1])
        j+=1
    
    print(audio_list,audio_list[106],audio_list[212],audio_list[159])
    idx0=audio_list.index(threshold[i][0])
    idx1=audio_list.index(threshold[i][1])
    
    p_list=[[],[],[]]
    '''for j in range(25):
        r, p = stats.pearsonr(lab_arr[:idx0],list(psd_arr[:idx0,j]))
        p_list[0].append(p)
    print(idx0,p_list[0])
    for j in range(25):
        r, p = stats.pearsonr(lab_arr[idx0:idx1],list(psd_arr[idx0:idx1,j]))
        p_list[1].append(p) 
    print(idx1-idx0,p_list[1])
    for j in range(25):
        r, p = stats.pearsonr(lab_arr[idx1:],list(psd_arr[idx1:,j]))
        p_list[2].append(p)
    print(319-idx1,p_list[2])
    print(sum(i<0.05 for i in p_list[0]),sum(i<0.05 for i in p_list[1]),sum(i<0.05 for i in p_list[2]))'''
    psd_3=[[],[],[]]
    for o in range(idx0):
        if lab_arr[o] < 3:
            psd_3[0].append(psd_arr[o,:])
        elif label[path] == 3:
            psd_3[1].append(psd_arr[o,:])
        else:
            psd_3[2].append(psd_arr[o,:])
    print(len(psd[0],len(psd[1]),len(psd[2]))
    p_list=[]
    for j in range(5): # all
        pvalue = str(round(stats.f_oneway(data_po[:, 4+j*5],data_nu[:, 4+j*5], data_ne[:, 4+j*5]).pvalue, 3))
        p_list.append(pvalue)
    print(p_list)
    